In [1]:
import pandas as pd
import glob, os

#Path for the data files and most of the variable declaration
path = "/Users/thieb/OneDrive - uzh.ch/Desktop/UNI/Master/Herbstsemester 2020/Value Creation/Data_dump_2" #"/Users/ankanghosh/Downloads/Semester III/Analytics & Value Creation/Data_dump"  #Change this path according your data
Net_Sales = ['SalesRevenueNet','SalesRevenueGoodsNet', 'RevenueFromContractWithCustomerExcludingAssessedTax']
Net_Income = ['NetIncomeLoss']
Operating_Profit = ['IncomeLossFromContinuingOperations', 'OperatingIncomeLoss', 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments']
RD_Expenditure = ['ResearchAndDevelopmentExpense', 'ResearchAndDevelopmentExpenseExcludingAcquiredInProcessCost']

companies = ['ABBOTT LABORATORIES']

In [2]:
#Loading all the csv files from the directory
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)

In [3]:
#Selection of the companies for the repective calculations
#df_selectedCompanies = concatenated_df[concatenated_df['name'] == 'ABBVIE INC.']  #For individual Companies

df_selectedCompanies = concatenated_df[concatenated_df['name'].isin(companies)]
df_selectedCompanies = df_selectedCompanies.drop(['version', 'cik', 'countryba', 'stprba', 'cityba', 'zipba', 'countryma', 'fy', 'filed', 'accepted', 'ddate', 'adsh'], axis=1)

#Seperating the columns based on tags 
df_netsales = df_selectedCompanies[df_selectedCompanies['tag'].isin(Net_Sales)].rename(columns = {'value':'NetSalesValues'})
df_netincome = df_selectedCompanies[df_selectedCompanies['tag'].isin(Net_Income)].rename(columns = {'value':'NetIncomeValues'})
df_operatingprofit = df_selectedCompanies[df_selectedCompanies['tag'].isin(Operating_Profit)].rename(columns ={'value':'OperatingProfit'})
df_rnd = df_selectedCompanies[df_selectedCompanies['tag'].isin(RD_Expenditure)].rename(columns ={'value':'RnDValues'})

In [4]:
df_netsales = df_netsales.sort_values(by=['qtrs'], ascending=False)
df_netsales.drop_duplicates(subset = 'period', 
                     keep = "first", inplace = True)
df_netsales = df_netsales.sort_values(by=['period'], ascending=True)

df_netincome = df_netincome.sort_values(by=['qtrs'], ascending=False)
df_netincome.drop_duplicates(subset = 'period', 
                     keep = "first", inplace = True)
df_netincome = df_netincome.sort_values(by=['period'], ascending=True)
df_operatingprofit = df_operatingprofit.sort_values(by=['qtrs'], ascending=False)
df_operatingprofit.drop_duplicates(subset = 'period', 
                     keep = "first", inplace = True)
df_operatingprofit = df_operatingprofit.sort_values(by=['period'], ascending=True)
df_rnd = df_rnd.sort_values(by=['qtrs'], ascending=False)
df_rnd.drop_duplicates(subset = 'period', 
                     keep = "first", inplace = True)
df_rnd = df_rnd.sort_values(by=['period'], ascending=True)

In [5]:
#Joining the tables based on adsh and period columns and calculating "Net Profit Margin"
df_NPM = pd.merge(df_netsales, df_netincome, on=["name", "period"])
df_NPM['Net_Profit_Margin']= df_NPM['NetIncomeValues']/df_NPM['NetSalesValues']


In [6]:
#Joining the tables based on adsh and period columns and calculating "Operating Profit Margin"
df_OPM = pd.merge(df_netsales, df_operatingprofit, on=["name", "period"])
df_OPM['Operational_Profit_Margin']= df_OPM['OperatingProfit']/df_OPM['NetSalesValues']

In [7]:
df_RCR = pd.merge(df_netsales, df_rnd, on=["name", "period"])
df_RCR['Return_on_Research_Capital_Ratio'] = df_RCR['RnDValues']/df_RCR['NetSalesValues']

In [8]:
df_SalesGrowth = df_netsales
df_SalesGrowth['SalesGrowth'] = df_SalesGrowth['NetSalesValues'].pct_change()
df_SalesGrowth['SalesGrowthYoY'] = df_SalesGrowth['NetSalesValues'].pct_change(periods = 4)

In [9]:
df_OperatingGrowth = df_operatingprofit
df_OperatingGrowth['OperatingGrowth'] = df_OperatingGrowth['OperatingProfit'].pct_change()
df_OperatingGrowth['OperatingGrowthYoY'] = df_OperatingGrowth['OperatingProfit'].pct_change(periods = 4)

In [10]:
df_DOL = pd.merge(df_SalesGrowth, df_OperatingGrowth, on=["name","period"])
df_DOL['DegreeOfOperatingLeverage'] = df_DOL['OperatingGrowth']/df_DOL['SalesGrowth']
df_DOL['DegreeOfOperatingLeverage'] = df_DOL['OperatingGrowthYoY']/df_DOL['SalesGrowthYoY']


In [11]:
df_NPM.drop([col for col in df_NPM.columns if col.endswith ('_y')],axis=1,inplace=True)
df_NPM.drop([col for col in df_NPM.columns if col.endswith ('_x')],axis=1,inplace=True)

df_OPM.drop([col for col in df_OPM.columns if col.endswith ('_y')],axis=1,inplace=True)
df_OPM.drop([col for col in df_OPM.columns if col.endswith ('_x')],axis=1,inplace=True)

df_RCR.drop([col for col in df_RCR.columns if col.endswith ('_y')],axis=1,inplace=True)
df_RCR.drop([col for col in df_RCR.columns if col.endswith ('_x')],axis=1,inplace=True)

df_SalesGrowth.drop([col for col in df_SalesGrowth.columns if col.endswith ('_y')],axis=1,inplace=True)
df_SalesGrowth.drop([col for col in df_SalesGrowth.columns if col.endswith ('_x')],axis=1,inplace=True)

df_OperatingGrowth.drop([col for col in df_OperatingGrowth.columns if col.endswith ('_y')],axis=1,inplace=True)
df_OperatingGrowth.drop([col for col in df_OperatingGrowth.columns if col.endswith ('_x')],axis=1,inplace=True)

df_DOL.drop([col for col in df_DOL.columns if col.endswith ('_y')],axis=1,inplace=True)
df_DOL.drop([col for col in df_DOL.columns if col.endswith ('_x')],axis=1,inplace=True)

df_Results = pd.merge(df_OPM, df_NPM, on = ["period", "name"])
df_Results = pd.merge(df_Results, df_RCR, on = ["period", "name"])
df_Results = pd.merge(df_Results, df_DOL, on = ["period", "name"])

df_Results.drop([col for col in df_Results.columns if col.endswith ('_y')],axis=1,inplace=True)
df_Results.drop([col for col in df_Results.columns if col.endswith ('_x')],axis=1,inplace=True)
df_Results.drop(['NetIncomeValues', 'RnDValues'], axis = 1, inplace = True)

df_Results.to_csv(r'C:\Users\thieb\OneDrive - uzh.ch\Desktop\UNI\Master\Herbstsemester 2020\Value Creation\Data_dump\Results_Abbott.csv')